In [73]:
%%writefile cuda_ray.cu
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define SPHERES 20

#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f
#define DIM 2048

struct Sphere {
    float r, b, g;
    float radius;
    float x, y, z;
    __device__ float hit(float ox, float oy, float* n) {
        float dx = ox - x;
        float dy = oy - y;
        if (dx * dx + dy * dy < radius * radius) {
            float dz = sqrtf(radius * radius - dx * dx - dy * dy);
            *n = dz / sqrtf(radius * radius);
            return dz + z;
        }
        return -INF;
    }
};

// using CUDA
__global__ void kernel (Sphere* s, unsigned char* ptr)
{
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;

    int offset = x + y * DIM;
    float ox = (x - DIM/2);
    float oy = (y - DIM/2);

    //printf("x:%d, y:%d, ox:%f, oy:%f\n",x,y,ox,oy);

    float r = 0, g = 0, b = 0;
    float maxz = -INF;
    for(int i = 0; i < SPHERES; i++) {
        float n;
        float t = s[i].hit(ox, oy, &n);
        if (t > maxz) {
            float fscale = n;
            r = s[i].r * fscale;
            g = s[i].g * fscale;
            b = s[i].b * fscale;
            maxz = t;
        }
    }

    ptr[offset * 4 + 0] = (int)(r * 255);
    ptr[offset * 4 + 1] = (int)(g * 255);
    ptr[offset * 4 + 2] = (int)(b * 255);
    ptr[offset * 4 + 3] = 255;
}

void ppm_write(unsigned char* bitmap, int xdim, int ydim, FILE* fp)
{
    int i, x, y;
    fprintf(fp, "P3\n");
    fprintf(fp, "%d %d\n", xdim, ydim);
    fprintf(fp, "255\n");
    for (y = 0; y < ydim; y++) {
        for (x = 0; x < xdim; x++) {
            i = x + y * xdim;
            fprintf(fp, "%d %d %d ", bitmap[4 * i], bitmap[4 * i + 1], bitmap[4 * i + 2]);
        }
        fprintf(fp, "\n");
    }
}


int main(int argc, char* argv[])
{
    unsigned char* bitmap;
    Sphere* dev_s;
    unsigned char* dev_b;

    srand(time(NULL));
    FILE* fp = fopen("result.ppm", "w");

    Sphere* temp_s = (Sphere*)malloc(sizeof(Sphere) * SPHERES);
    for (int i = 0; i < SPHERES; i++) {
        temp_s[i].r = rnd(1.0f);
        temp_s[i].g = rnd(1.0f);
        temp_s[i].b = rnd(1.0f);
        temp_s[i].x = rnd(2000.0f) - 1000;
        temp_s[i].y = rnd(2000.0f) - 1000;
        temp_s[i].z = rnd(2000.0f) - 1000;
        temp_s[i].radius = rnd(200.0f) + 40;
    }

    cudaMalloc((void**)&dev_s, SPHERES * sizeof(Sphere));
    cudaMalloc((void**)&dev_b, sizeof(unsigned char) * DIM * DIM * 4);

    cudaMemcpy(dev_s, temp_s, sizeof(Sphere) * SPHERES, cudaMemcpyHostToDevice);

    clock_t start_time = clock();

    // number of blocks, threads per block
    // it computes using CUDA and get output of ppm image
    kernel<<<dim3((DIM + 15) / 16, (DIM + 15) / 16), dim3(16, 16)>>>(dev_s, dev_b);

    bitmap = (unsigned char*)malloc(sizeof(unsigned char) * DIM * DIM * 4);
    cudaMemcpy(bitmap, dev_b, sizeof(unsigned char) * DIM * DIM * 4, cudaMemcpyDeviceToHost);

    clock_t end_time = clock();
    clock_t diff_time = end_time - start_time;
	  printf("CUDA ray tracing: %f sec. \n", (double)diff_time/CLOCKS_PER_SEC);

    ppm_write(bitmap, DIM, DIM, fp);
    printf("[%s] was generated.\n", "result.ppm");

    fclose(fp);
    free(bitmap);
    free(temp_s);
    cudaFree(dev_s); cudaFree(dev_b);
    return 0;
}

Overwriting cuda_ray.cu


In [74]:
!nvcc cuda_ray.cu

In [75]:
!./a.out

CUDA ray tracing: 0.012977 sec. 
[result.ppm] was generated.


In [69]:
%%writefile openmp_ray.cpp
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <omp.h>

#define SPHERES 20

#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f
#define DIM 2048

struct Sphere {
    float r, b, g;
    float radius;
    float x, y, z;
    float hit(float ox, float oy, float* n) {
        float dx = ox - x;
        float dy = oy - y;
        if (dx * dx + dy * dy < radius * radius) {
            float dz = sqrtf(radius * radius - dx * dx - dy * dy);
            *n = dz / sqrtf(radius * radius);
            return dz + z;
        }
        return -INF;
    }
};

void kernel(int x, int y, Sphere* s, unsigned char* ptr)
{
    int offset = x + y * DIM;
    float ox = (x - DIM/2);
    float oy = (y - DIM/2);

    //printf("x:%d, y:%d, ox:%f, oy:%f\n",x,y,ox,oy);

    float r = 0, g = 0, b = 0;
    float maxz = -INF;
    for(int i = 0; i < SPHERES; i++) {
        float n;
        float t = s[i].hit(ox, oy, &n);
        if (t > maxz) {
            float fscale = n;
            r = s[i].r * fscale;
            g = s[i].g * fscale;
            b = s[i].b * fscale;
            maxz = t;
        }
    }

    ptr[offset * 4 + 0] = (int)(r * 255);
    ptr[offset * 4 + 1] = (int)(g * 255);
    ptr[offset * 4 + 2] = (int)(b * 255);
    ptr[offset * 4 + 3] = 255;
}

void ppm_write(unsigned char* bitmap, int xdim, int ydim, FILE* fp)
{
    int i, x, y;
    fprintf(fp, "P3\n");
    fprintf(fp, "%d %d\n", xdim, ydim);
    fprintf(fp, "255\n");
    for (y = 0; y < ydim; y++) {
        for (x = 0; x < xdim; x++) {
            i = x + y * xdim;
            fprintf(fp, "%d %d %d ", bitmap[4 * i], bitmap[4 * i + 1], bitmap[4 * i + 2]);
        }
        fprintf(fp, "\n");
    }
}


int main(int argc, char* argv[])
{
    int no_threads;
    int x,y;
    unsigned char* bitmap;

    srand(time(NULL));
    if (argc!=2) {
        printf("> openmp_ray [option]\n");
        printf("[option] 1~16: OpenMP using 1~16 threads\n");
        printf("for example, '> openmp_ray 8' means executing OpenMP with 8 threads\n");
        exit(0);
	  }
    FILE* fp = fopen("result.ppm", "w");

    no_threads = atoi(argv[1]);

    Sphere* temp_s = (Sphere*)malloc(sizeof(Sphere) * SPHERES);
    for (int i = 0; i < SPHERES; i++) {
        temp_s[i].r = rnd(1.0f);
        temp_s[i].g = rnd(1.0f);
        temp_s[i].b = rnd(1.0f);
        temp_s[i].x = rnd(2000.0f) - 1000;
        temp_s[i].y = rnd(2000.0f) - 1000;
        temp_s[i].z = rnd(2000.0f) - 1000;
        temp_s[i].radius = rnd(200.0f) + 40;
    }

    bitmap = (unsigned char*)malloc(sizeof(unsigned char) * DIM * DIM * 4);

    // check timer with clock_k
    clock_t start_time = clock();

    // using openMP multithreading
#pragma omp parallel for schedule(dynamic) num_threads(no_threads)
    for (y = 0; y < DIM; y++) {
        for (x=0;x<DIM;x++) {
            kernel(x, y, temp_s, bitmap);
        }
    }

    clock_t end_time = clock();
    clock_t diff_time = end_time - start_time;
	  printf("OpenMP (%d threads) ray tracing: %.3lf sec. \n", no_threads, (double)diff_time/CLOCKS_PER_SEC);

    ppm_write(bitmap, DIM, DIM, fp);
    printf("[%s] was generated.\n", "result.ppm");

    fclose(fp);
    free(bitmap);
    free(temp_s);

    return 0;
}

Overwriting openmp_ray.cpp


In [70]:
!nvcc openmp_ray.cpp

In [72]:
!./a.out 16

OpenMP (16 threads) ray tracing: 1.000 sec. 
[result.ppm] was generated.


In [76]:
%%writefile raytracing.cpp
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define SPHERES 20

#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f
#define DIM 2048

struct Sphere {
    float   r,b,g;
    float   radius;
    float   x,y,z;
    float hit( float ox, float oy, float *n ) {
        float dx = ox - x;
        float dy = oy - y;
        if (dx*dx + dy*dy < radius*radius) {
            float dz = sqrtf( radius*radius - dx*dx - dy*dy );
            *n = dz / sqrtf( radius * radius );
            return dz + z;
        }
        return -INF;
    }
};

void kernel(int x, int y, Sphere* s, unsigned char* ptr)
{
	int offset = x + y*DIM;
	float ox = (x - DIM/2);
	float oy = (y - DIM/2);

	//printf("x:%d, y:%d, ox:%f, oy:%f\n",x,y,ox,oy);

	float r=0, g=0, b=0;
	float   maxz = -INF;
	for(int i=0; i<SPHERES; i++) {
		float   n;
		float   t = s[i].hit( ox, oy, &n );
		if (t > maxz) {
			float fscale = n;
			r = s[i].r * fscale;
			g = s[i].g * fscale;
			b = s[i].b * fscale;
			maxz = t;
		}
	}

	ptr[offset*4 + 0] = (int)(r * 255);
	ptr[offset*4 + 1] = (int)(g * 255);
	ptr[offset*4 + 2] = (int)(b * 255);
	ptr[offset*4 + 3] = 255;
}

void ppm_write(unsigned char* bitmap, int xdim,int ydim, FILE* fp)
{
	int i,x,y;
	fprintf(fp,"P3\n");
	fprintf(fp,"%d %d\n",xdim, ydim);
	fprintf(fp,"255\n");
	for (y=0;y<ydim;y++) {
		for (x=0;x<xdim;x++) {
			i=x+y*xdim;
			fprintf(fp,"%d %d %d ",bitmap[4*i],bitmap[4*i+1],bitmap[4*i+2]);
		}
		fprintf(fp,"\n");
	}
}

int main(int argc, char* argv[])
{
	int no_threads;
	int option;
	int x,y;
	unsigned char* bitmap;

	srand(time(NULL));
  FILE* fp = fopen("result.ppm", "w");

	Sphere *temp_s = (Sphere*)malloc( sizeof(Sphere) * SPHERES );
	for (int i=0; i<SPHERES; i++) {
		temp_s[i].r = rnd( 1.0f );
		temp_s[i].g = rnd( 1.0f );
		temp_s[i].b = rnd( 1.0f );
		temp_s[i].x = rnd( 2000.0f ) - 1000;
		temp_s[i].y = rnd( 2000.0f ) - 1000;
		temp_s[i].z = rnd( 2000.0f ) - 1000;
		temp_s[i].radius = rnd( 200.0f ) + 40;
	}

	bitmap=(unsigned char*)malloc(sizeof(unsigned char)*DIM*DIM*4);

  clock_t start_time = clock();

	for (x=0;x<DIM;x++)
		for (y=0;y<DIM;y++) kernel(x,y,temp_s,bitmap);
	ppm_write(bitmap,DIM,DIM,fp);

  clock_t end_time = clock();
  clock_t diff_time = end_time - start_time;
  printf("OpenMP (%d threads) ray tracing: %.3lf sec. \n", no_threads, (double)diff_time/CLOCKS_PER_SEC);

	fclose(fp);
	free(bitmap);
	free(temp_s);

	return 0;
}

Overwriting raytracing.cpp


In [77]:
!nvcc raytracing.cpp

In [78]:
!./a.out

OpenMP (21979 threads) ray tracing: 1.598 sec. 


In [79]:
%%writefile thrust_ex.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>

#include <thrust/transform.h>
#include <thrust/sequence.h>

#include <stdio.h>
#include <time.h>
#include <iostream>

long num_steps = 200000;
double step = 1.0/(double) num_steps;

template<typename T>
struct integral
{
    double step;

    integral(double step) : step(step){}

    __host__ __device__
        T operator()(const T &i) const {
            double x = (i+0.5)*step;
            return 4.0/(1.0+x*x);
        }
};

int main(){
    clock_t start_time = clock();
    thrust::device_vector<int> index(num_steps);

    // make new sequence
    thrust::sequence(index.begin(), index.end());

    integral<double> unary_op(step);
    thrust::plus<double> binary_op;
    double init = 0.0;

    // transform (using function unary_op, set by 'integral') and reduction (reduce to a single value, set by summation)
    double sum = thrust::transform_reduce(index.begin(), index.end(), unary_op, init, binary_op);
    double pi = step * sum;
    printf("pi=%.10lf\n",pi);

    clock_t end_time = clock();

    clock_t diff_time = end_time - start_time;
	  printf("execution time: %.3lf sec. \n", (double)diff_time/CLOCKS_PER_SEC);

    return 0;
}

Writing thrust_ex.cu


In [80]:
!nvcc thrust_ex.cu

In [81]:
!./a.out

pi=3.1415926536
execution time: 0.721 sec. 


In [82]:
%%writefile omp_pi_one.c
#include <omp.h>

#include <time.h>
#include <stdio.h>

long num_steps = 1000000000;
double step;

void main ()
{
	long i; double x, pi, sum = 0.0;

	clock_t start_time = clock();
	step = 1.0/(double) num_steps;
	for (i=0;i< num_steps; i++){
		x = (i+0.5)*step;
		sum = sum + 4.0/(1.0+x*x);
	}
	pi = step * sum;
	clock_t end_time = clock();

  clock_t diff_time = end_time - start_time;
  printf("execution time: %.3lf sec. \n", (double)diff_time/CLOCKS_PER_SEC);

	printf("pi=%.10lf\n",pi);
}

Overwriting omp_pi_one.c


In [83]:
!nvcc omp_pi_one.c

In [84]:
!./a.out

execution time: 8.579 sec. 
pi=3.1415926536
